# Radiation Therapy Treatment Optimization using Gurobi

In this example, we will model and solve a radiation therapy treatment optimization problem using Gurobi in Python. The objective is to minimize radiation exposure to healthy tissues while ensuring sufficient radiation dosage in tumor regions for cancer treatment.

**Problem Description:**

- **Decision Variables:**
  - $x_1$, $x_2$: Continuous variables representing treatment intensities or geometries.
  - **Binary variables (0/1):** Represent placement decisions (e.g., whether to place a radioactive seed in a specific location in brachytherapy).

- **Objective Function:**
  $$
  \text{Minimize } Z = 0.4x_1 + 0.5x_2
  $$
  Where $Z$ represents a weighted sum of treatment impacts (cost or damage to healthy tissues).

- **Constraints:**
  $$
  0.3x_1 + 0.1x_2 \leq 2.7
  $$
  $$
  0.5x_1 + 0.5x_2 = 6
  $$
  $$
  0.6x_1 + 0.4x_2 \geq 6
  $$
  - Non-negativity constraints: $x_1 \geq 0$, $x_2 \geq 0$.

This model is an example of a **mixed-integer programming** (MIP) problem, where continuous and binary decision variables are used together to optimize a medical treatment plan. This type of model is especially suitable for automated systems that need to make discrete decisions, such as placement of seeds in brachytherapy.

## Step 1: Import Gurobi and Set Up the Environment

Gurobi is a powerful optimization solver that can handle linear programming, mixed-integer programming, and other types of mathematical optimization problems. We import the `gurobipy` module to access its functions and classes.


In [1]:
import gurobipy as gp
from gurobipy import GRB

## Step 2: Create a New Model

We create a new model object where we will add our decision variables, objective function, and constraints. Naming the model helps in identifying it, especially when dealing with multiple models.


In [2]:
# Create a new model
model = gp.Model('Radiation_Therapy_Optimization')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-31


## Step 3: Define Decision Variables

We define our decision variables:

- Continuous variables \( x_1 \) and \( x_2 \) representing treatment intensities.
- Binary variables \( y_1 \) and \( y_2 \) representing placement decisions (e.g., whether to place a radioactive seed in a specific location).

The continuous variables have a lower bound of 0. The binary variables can take values of 0 or 1.

In [3]:
# Add continuous decision variables
x1 = model.addVar(name='x1', vtype=GRB.CONTINUOUS, lb=0)
x2 = model.addVar(name='x2', vtype=GRB.CONTINUOUS, lb=0)

# Add binary decision variables
y1 = model.addVar(name='y1', vtype=GRB.BINARY)
y2 = model.addVar(name='y2', vtype=GRB.BINARY)

## Step 4: Set the Objective Function

The objective function aims to minimize the weighted sum of treatment impacts on healthy tissues:

\[
\text{Minimize } Z = 0.4x_1 + 0.5x_2
\]

We set this objective in the model using the `setObjective` method, specifying `GRB.MINIMIZE` to indicate that we are minimizing the objective.

In [4]:
# Set objective function
model.setObjective(0.4 * x1 + 0.5 * x2, GRB.MINIMIZE)

## Step 5: Add Constraints

We add the constraints to the model using the `addConstr` method.

### Constraint 1:

$$
0.3x_1 + 0.1x_2 \leq 2.7
$$

This constraint limits the exposure to a particular sensitive area.

### Constraint 2:

$$
0.5x_1 + 0.5x_2 = 6
$$

This constraint ensures that the total treatment intensity meets a required level.

### Constraint 3:

$$
0.6x_1 + 0.4x_2 \geq 6
$$

This constraint ensures sufficient radiation in the tumor region.

### Linking Continuous and Binary Variables:

If we want to model that the continuous variables $x_i$ can only be positive if the corresponding binary variable $y_i$ is 1 (i.e., treatment is applied), we can add the following constraints:

$$
x_i \leq M y_i \quad \text{for } i = 1, 2
$$

Where $M$ is a sufficiently large constant.

Assuming $M = 10$, we add these constraints to link $x_i$ and $y_i$.

In [5]:
# Add constraints
model.addConstr(0.3 * x1 + 0.1 * x2 <= 2.7, name='Constraint1')
model.addConstr(0.5 * x1 + 0.5 * x2 == 6, name='Constraint2')
model.addConstr(0.6 * x1 + 0.4 * x2 >= 6, name='Constraint3')

# Linking constraints between x_i and y_i
M = 10  # Large constant
model.addConstr(x1 <= M * y1, name='Linking_x1_y1')
model.addConstr(x2 <= M * y2, name='Linking_x2_y2')

<gurobi.Constr *Awaiting Model Update*>

## Step 6: Optimize the Model

With the model fully defined, we call the `optimize` method to solve the optimization problem. Gurobi uses advanced algorithms to find the optimal solution efficiently.

In [6]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.1.0 24B5046f)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 4 columns and 10 nonzeros
Model fingerprint: 0xe9022a0c
Variable types: 2 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [4e-01, 5e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 6e+00]
Presolve removed 5 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 5.25 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.250000000000e+00, best bound 5.250000000000e+00, gap 0.0000%


## Step 7: Display the Results

After the optimization process, we can access the optimal values of the decision variables and the objective function value. We check if the model status is `GRB.OPTIMAL` to ensure that an optimal solution was found before attempting to display the results.

In [7]:
# Check if the model has an optimal solution
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    print(f"  x1 (Treatment intensity 1) = {x1.x}")
    print(f"  x2 (Treatment intensity 2) = {x2.x}")
    print(f"  y1 (Placement decision 1) = {y1.x}")
    print(f"  y2 (Placement decision 2) = {y2.x}")
    print(f"  Minimum impact (Z) = {model.objVal}")
else:
    print("No optimal solution found.")

Optimal solution found:
  x1 (Treatment intensity 1) = 7.5
  x2 (Treatment intensity 2) = 4.5
  y1 (Placement decision 1) = 1.0
  y2 (Placement decision 2) = 1.0
  Minimum impact (Z) = 5.25


## Interpretation of Results

The optimal solution provides the treatment intensities and placement decisions that minimize the impact on healthy tissues while ensuring sufficient radiation dosage to tumor regions. The binary variables $ y_1 $ and  $ y_2 $ indicate whether the treatment is applied in specific locations.